In [1]:
import numpy as np 
import pandas as pd
from tqdm import tqdm

import sys
sys.path.append('../')
from retinanet import coco_eval
from retinanet import csv_eval
from retinanet import model
from retinanet import paps_eval
from retinanet import paps_train

# from retinanet import retina
from retinanet.dataloader import *
from retinanet.anchors import Anchors
from retinanet.anchors import *
from retinanet.losses import *
from retinanet.scheduler import *
from retinanet.hourglass import hg1, hg2, hg8
from retinanet.parallel import DataParallelModel, DataParallelCriterion

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler
import torch.optim as optim

# python train_paps.py --start_epoch 0 --end_epoch 120 --batch_size 24 \
#                     --saved_dir $OUT_MODEL_DIR --gpu_num 0 --num_workers 12 \
#                     --target_threshold 7 --topk 20 --filter_option 1  | 2>&1 | tee $log


In [2]:
# device = torch.device('cpu')
# device = torch.device('cuda')
GPU_NUM = 0 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
model = hg2(device, pretrained=True, progress=False, num_classes=2)
model.to(device)

num_classes 2
num_anchors per feature map 1


HourglassNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
  )
  (layer2): Sequential(
    (0): Bottleneck(
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=Tr

In [3]:
# model

In [4]:

# train_info = np.load('../data/train.npy', allow_pickle=True, encoding='latin1').item()
# train_info

batch_size = 32
dataset_train = PapsDataset('../data/', set_name='train_2class',
                            transform=train_transforms)

train_data_loader = DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True,
    num_workers=12,
#    pin_memory=True,
#    prefetch_factor=1,
    collate_fn=collate_fn
    
)

loading annotations into memory...
Done (t=0.28s)
creating index...
index created!


In [5]:
filter_option = 6
criterion = PapsLoss(device, target_threshold=8, topk=10, filter_option=filter_option)
if filter_option == 6 :
    ispimage = True
else :
    ispimage = False
criterion = criterion.to(device)
model.training = True
print(ispimage)

# https://gaussian37.github.io/dl-pytorch-lr_scheduler/
optimizer = optim.Adam(model.parameters(), lr = 1e-8)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=20, T_mult=2, eta_max=0.0008,  T_up=5, gamma=0.5)
# CosineAnnealingWarmRestarts

True


In [6]:
saved_dir = '../trained_models/HourGlass/loss_image_filter' + str(0) + '/'
s_epoch= 0
e_epoch= 60

In [7]:
if os.path.isdir(saved_dir) == False :
    print('folder is made')
    os.makedirs(saved_dir)

In [8]:
if os.path.isfile(saved_dir + 'epoch_' + str(e_epoch) +'_model.pt') :
    print('pretrainind file loading')
    state = torch.load(saved_dir + 'epoch_' + str(e_epoch) +'_model.pt')
    epoch = state['epoch']
    model.load_state_dict(state['state_dict'], strict=False)
    optimizer.load_state_dict(state['optimizer'])
    last_loss = state['loss']
else :
    last_loss = 0.6

pretrainind file loading


In [9]:
paps_train.train_paps(dataloader=train_data_loader, 
                      model=model, 
                      criterion=criterion,
                      saved_dir=saved_dir, 
                      optimizer=optimizer,
                      scheduler=scheduler,
                      device = device,
                      s_epoch= s_epoch,
                      e_epoch= e_epoch,
                      last_loss = last_loss,
                      ispimage = ispimage)  

  0%|          | 0/465 [00:10<?, ?it/s, avg_loss=0.165, batch_loss=0.165, cls_loss=0.145, lr=0.0002, num_det=tensor(337, device='cuda:0'), reg_loss=0.0205]

torch.Size([32, 1600])


  0%|          | 1/465 [00:11<1:25:23, 11.04s/it, avg_loss=0.165, batch_loss=0.165, cls_loss=0.145, lr=0.0002, num_det=tensor(337, device='cuda:0'), reg_loss=0.0205]

torch.Size([32, 1600])


  0%|          | 2/465 [00:12<42:01,  5.45s/it, avg_loss=0.165, batch_loss=0.165, cls_loss=0.145, lr=0.0002, num_det=tensor(337, device='cuda:0'), reg_loss=0.0205]  

...

torch.Size([32, 1600])


torch.Size([32, 1600])


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-9b2a9e1d1f91>", line 11, in <module>
    ispimage = ispimage)
  File "../retinanet/paps_train.py", line 72, in train_paps
    beta, pimages)
  File "/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "../retinanet/losses.py", line 319, in forward
    elif int(self.filter_option) == 6 and pimage != None :
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError

TypeError: object of type 'NoneType' has no len()

In [ ]:
# state = {
#     'epoch': 0,
#     'state_dict': model.state_dict(),
#     'optimizer': optimizer.state_dict(),
#     'loss' : 0.6
# }
# torch.save(state, saved_dir + 'model.pt')

In [9]:
dataset_val = PapsDataset('../data/', set_name='val_2class',
                            transform=val_transforms)

val_data_loader = DataLoader(
    dataset_val,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


In [10]:
paps_eval.evaluate_paps(dataset=dataset_val, 
  dataloader=val_data_loader, 
  model=model, 
  saved_dir=saved_dir, 
  device = device,
  threshold=0.1) 


100%|██████████| 3739/3739 [02:19<00:00, 26.87it/s]


GT_results 17716
pred_results 94386
validation time : 140.37444257736206
Loading and preparing results...
DONE (t=0.38s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.17s)
creating index...
index created!
******************total*********************
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=11.31s).
Accumulating evaluation results...
DONE (t=0.52s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.019
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.074
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.014
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.177
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.035
 Average Recall     (AR) @[

In [ ]:
criterion = PapsLoss(device, target_threshold=9, topk=10, filter_option=1)
criterion = criterion.to(device)
s_epoch= 60
e_epoch= 120


paps_train.train_paps(dataloader=train_data_loader, 
                      model=model, 
                      criterion=criterion,
                      saved_dir=saved_dir, 
                      optimizer=optimizer,
                      scheduler=scheduler,
                      device = device,
                      s_epoch= s_epoch,
                      e_epoch= e_epoch,
                      last_loss = last_loss)  

  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                

60th epochs loss 0.18394588058193526 lr 0.0002 


  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                 

61th epochs loss 0.11387616698431154 lr 0.000160008 
best model is saved


  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                 

...

  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                  

108th epochs loss 0.08595628944090289 lr 5.1379750628648584e-05 


  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                  

109th epochs loss 0.08537890671346432 lr 3.820564620998241e-05 


 72%|███████▏  | 333/465 [04:29<01:19,  1.66it/s, avg_loss=0.0855, batch_loss=0.0715, cls_loss=0.0607, lr=2.68e-5, num_det=tensor(65, device='cuda:0'), reg_loss=0.0108] 

In [ ]:
paps_eval.evaluate_paps(dataset=dataset_val, 
  dataloader=val_data_loader, 
  model=model, 
  saved_dir=saved_dir, 
  device = device,
  threshold=0.1) 
